In [2]:
import os,sys
import cv2
from tqdm import tqdm
import re
import random

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical


In [3]:
from tensorflow.keras.applications.vgg16 import preprocess_input

In [4]:
path = '/kaggle/input/stanford-dogs-dataset-traintest/cropped/train'


## old

In [4]:
# def create_x_y(label,directory):
#     for img in tqdm(os.listdir(directory)): # if you like the loading bars :D
# #     for img in os.listdir(directory):
#         path = os.path.join(directory,img)
#         img = cv2.imread(path, cv2.IMREAD_COLOR)
        
#         X.append(np.array(img))
#         Z.append(str(label)) # raw label

In [5]:
# # path = '/Users/faskerrr/code/Faskerrr/Doggos-101/raw_data/stanford_dogs_pics/cropped/train/' # path for local data
# breeds = os.listdir(path)
# # breeds

In [6]:
# data_dirs = [path + '/' + i + '/' for i in breeds if i != '.DS_Store']
# # data_dirs

In [7]:
# breed_names = [re.findall('n\d{8}-(.*)', i)[0][:-1].capitalize() for i in data_dirs]
# # breed_names

In [8]:
# X = []
# Z = []

# for i in range(120):
#     create_x_y(breed_names[i], data_dirs[i])
    

In [9]:
# X = np.array(X)
# # X

In [10]:
# from tensorflow.keras.applications.vgg16 import preprocess_input

In [11]:
# X_prep = preprocess_input(X)

In [12]:
# X_prep.shape

In [13]:
# np.save('data.npy', X_prep) # save

In [14]:
# label_encoder= LabelEncoder()
# y = label_encoder.fit_transform(Z)
# y = to_categorical(y,120)

In [5]:
from tensorflow.keras.utils import image_dataset_from_directory
data_dir = path

train_ds = image_dataset_from_directory(
    data_dir,
    labels='inferred',
    label_mode='categorical',
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(224, 224),
    batch_size=32)

Found 12000 files belonging to 120 classes.
Using 9600 files for training.


In [6]:
validation_ds = image_dataset_from_directory(
    data_dir,
    labels='inferred',
    label_mode='categorical',
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(224, 224),
    batch_size=32)

Found 12000 files belonging to 120 classes.
Using 2400 files for validation.


In [7]:
# from tensorflow.keras.applications.vgg16 import preprocess_input

In [8]:
prep_train_ds = train_ds.map(lambda x, y: (preprocess_input(x), y))
prep_val_ds = validation_ds.map(lambda x, y: (preprocess_input(x), y))

In [9]:
from tensorflow.keras.applications.vgg16 import VGG16

In [15]:
vgg = VGG16(include_top=False, weights='imagenet',
            input_shape=(224,224,3),
            
            )
vgg.trainable = False

58889256/58889256 [==============================] - 2s 0us/step


In [16]:
def model_init():
    model = Sequential([
    layers.RandomFlip("horizontal", input_shape=(224,224,3)),
    layers.RandomZoom(0.1),
    layers.RandomTranslation(0.2, 0.2),
    vgg,
    # layers.Flatten(),
    # layers.Dense(100, activation='relu'),
    # layers.Dense(100, activation='relu'),
    layers.Flatten(),
#     layers.Dense(64, activation='relu'),
#     layers.Dropout(0.5),
    layers.Dense(120, activation='softmax')
    ])
    return model

In [23]:
vgg_model = model_init()
vgg_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_flip_2 (RandomFlip)  (None, 224, 224, 3)       0         
                                                                 
 random_zoom_2 (RandomZoom)  (None, 224, 224, 3)       0         
                                                                 
 random_translation_2 (Rando  (None, 224, 224, 3)      0         
 mTranslation)                                                   
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_2 (Flatten)         (None, 25088)             0         
                                                                 
 dense_2 (Dense)             (None, 120)               3010680   
                                                      

In [24]:
opt = optimizers.Adam(learning_rate = 0.010)

def compile_model(model):
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [25]:
vgg_model = compile_model(vgg_model)

MODEL = "model_1"

modelCheckpooint = ModelCheckpoint("{}.h5".format(MODEL), monitor="val_loss", verbose=0, save_best_only=True)

LRreducer = ReduceLROnPlateau(monitor="val_loss", factor = 0.1, patience=3, verbose=1, min_lr=0)

es = EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)

history = vgg_model.fit(prep_train_ds,
                       validation_data=prep_val_ds,
                       epochs=10,
                       batch_size=64,
                       callbacks=[es, LRreducer],
                       verbose=1
                      )

Epoch 1/10
300/300 [==============================] - 100s 322ms/step - loss: 57.4234 - accuracy: 0.2979 - val_loss: 43.7527 - val_accuracy: 0.5288 - lr: 0.0010
Epoch 2/10
300/300 [==============================] - 99s 329ms/step - loss: 37.8152 - accuracy: 0.5686 - val_loss: 52.3781 - val_accuracy: 0.5383 - lr: 0.0010
Epoch 3/10
300/300 [==============================] - 100s 333ms/step - loss: 32.5530 - accuracy: 0.6480 - val_loss: 57.5157 - val_accuracy: 0.5796 - lr: 0.0010
Epoch 4/10
300/300 [==============================] - ETA: 0s - loss: 27.9316 - accuracy: 0.7134
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
300/300 [==============================] - 97s 323ms/step - loss: 27.9316 - accuracy: 0.7134 - val_loss: 59.3957 - val_accuracy: 0.5788 - lr: 0.0010
Epoch 5/10
  5/300 [..............................] - ETA: 1:27 - loss: 24.7990 - accuracy: 0.7812

KeyboardInterrupt: 

## Test

In [ ]:
test_dir = "/kaggle/input/stanford-dogs-dataset-traintest/cropped/test"


test_ds = image_dataset_from_directory(
    test_dir,
    labels='inferred',
    label_mode='categorical',
    image_size=(224, 224),
    batch_size=32)

prep_test_ds = test_ds.map(lambda x, y: (preprocess_input(x), y))

loss, accuracy = vgg_model.evaluate(prep_test_ds)
print(f"Test loss: {loss:.4f}")
print(f"Test accuracy: {accuracy:.4f}")

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(20)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()